In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
import lib.preprocess.user_load_data as user_load_data
import lib.preprocess.pipline as pipline
import importlib

importlib.reload(user_load_data)
importlib.reload(pipline)

<module 'lib.preprocess.pipline' from 'D:\\Project\\Python\\household-electricity\\lib\\preprocess\\pipline.py'>

In [4]:
# 讀取原始 .csv 檔
original_dataSet = user_load_data.load_dataset('2018-08-24-01-15-29_table_0502.csv',
                                               dtype={ 'userId': str, 'channelId': int })
original_dataSet = user_load_data.transform_time(original_dataSet,
                                                 'reportTime', format='%Y-%m-%d %H:%M:%S')
original_dataSet.head()

,userId,channelId,reportTime,w
0,39,0,2018-03-01 00:00:52,1176.0
1,39,0,2018-03-01 00:01:52,1176.0
2,39,0,2018-03-01 00:02:52,1081.0
3,39,0,2018-03-01 00:03:52,1170.0
4,39,0,2018-03-01 00:04:52,1105.0


In [5]:
original_dataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39436913 entries, 0 to 39436912
Data columns (total 4 columns):
userId        object
channelId     int32
reportTime    datetime64[ns]
w             float64
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 1.0+ GB


In [6]:
original_dataSet.describe()

,channelId,w
count,3.943691e+07,3.943691e+07
mean,3.055408e+00,1.220411e+02
std,1.988076e+00,1.475831e+03
min,0.000000e+00,-6.026000e+03
25%,2.000000e+00,0.000000e+00
50%,3.000000e+00,4.000000e+00
75%,5.000000e+00,1.000000e+02
max,6.000000e+00,1.677440e+05


In [7]:
# 先做 channelId 0
channelId_0_dataSet = pipline.only_use_channelId_0_dataSet(original_dataSet)
channelId_0_dataSet.describe()

,channelId,w
count,7814164.0,7.814164e+06
mean,0.0,4.340094e+02
std,0.0,5.213377e+02
min,0.0,-6.026000e+03
25%,0.0,1.800000e+02
50%,0.0,3.030000e+02
75%,0.0,4.900000e+02
max,0.0,1.673200e+04


In [8]:
channelId_0_dataSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7814164 entries, 0 to 39378856
Data columns (total 4 columns):
userId        object
channelId     int32
reportTime    datetime64[ns]
w             float64
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 268.3+ MB


In [9]:
user_load_data.save_csv(channelId_0_dataSet, 'channelId_0_dataSet.csv')

save: data/2018-08-28-20-51-40_channelId_0_dataSet.csv


In [10]:
# 刪除異常值，因為發現 sensor 本身有問題
delete_outliers_dataSet = pipline.delete_outliers_dataSet(channelId_0_dataSet)
delete_outliers_dataSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7785347 entries, 0 to 39378856
Data columns (total 4 columns):
userId        object
channelId     int32
reportTime    datetime64[ns]
w             float64
dtypes: datetime64[ns](1), float64(1), int32(1), object(1)
memory usage: 267.3+ MB


D:\Project\Python\household-electricity\preprocess\user_load_data.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  dataeSet[column] = pd.to_datetime(dataeSet[column], format=format)


In [11]:
user_load_data.save_csv(delete_outliers_dataSet, 'delete_outliers_dataSet.csv')

save: data/2018-08-28-21-00-43_delete_outliers_dataSet.csv


In [12]:
# 以 userId 分類，彙整每個使用者用電資料為每 15 分鐘一筆，w 四捨五入至小數 2 位
group_dataSet = pipline.group_dataSet(delete_outliers_dataSet)
group_dataSet.head()

,userId,reportTime,w
0,1,2017-08-01 00:00:00,907.80
1,1,2017-08-01 00:15:00,701.20
2,1,2017-08-01 00:30:00,964.20
3,1,2017-08-01 00:45:00,817.75
4,1,2017-08-01 01:00:00,813.60


In [13]:
user_load_data.save_csv(group_dataSet, 'group_dataSet.csv')

save: data/2018-08-28-21-01-56_group_dataSet.csv


In [14]:
# 彙整與轉置多個使用者的用電資料 (96 期)
consolidation_dataSet = pipline.consolidation_dataSet(group_dataSet)
consolidation_dataSet.head()

process userId1	265
process userId10	361
process userId11	509
process userId12	657
process userId13	805
process userId14	953
process userId15	1101
process userId16	1249
process userId17	1397
process userId18	1544
process userId19	1692
process userId2	1965
process userId20	2113
process userId21	2261
process userId22	2409
process userId23	2557
process userId24	2705
process userId25	2853
process userId26	3001
process userId27	3149
process userId28	3297
process userId29	3445
process userId3	3718
process userId30	3866
process userId31	4014
process userId32	4162
process userId33	4307
process userId34	4455
process userId35	4603
process userId36	4751
process userId37	4899
process userId38	5031
process userId39	5092
process userId4	5365
process userId40	5426
process userId41	5487
process userId42	5540
process userId43	5596
process userId44	5744
process userId45	5797
process userId46	5858
process userId47	5912
process userId48	5962
process userId49	6023
process userId5	6296
process userId50	6356

,uuid,userId,reportTime,period_1,period_2,period_3,period_4,period_5,period_6,period_7,...,period_87,period_88,period_89,period_90,period_91,period_92,period_93,period_94,period_95,period_96
0,1020170801,1,2017-08-01,907.80,701.2,964.2,817.75,813.6,1037.8,723.2,...,1577.2,4020.0,4583.25,1840.6,1320.40,1093.4,907.0,893.0,906.8,469.75
1,1020170802,1,2017-08-02,693.80,694.8,947.6,1022.60,1286.6,848.6,1360.0,...,1818.2,1599.2,1741.60,1537.0,1150.20,1271.0,1236.4,1567.8,1151.8,1392.60
2,1020170803,1,2017-08-03,974.25,1343.2,1513.0,844.80,1148.4,1326.2,1319.8,...,1571.0,4096.6,1514.00,902.0,797.00,962.8,791.4,951.4,884.2,734.80
3,1020170804,1,2017-08-04,1090.25,979.6,699.8,914.20,674.6,676.8,752.6,...,5295.8,3434.0,2591.60,3004.6,2508.75,1623.0,1172.2,1183.8,889.0,1394.00
4,1020170805,1,2017-08-05,1080.50,1151.0,1109.6,883.20,1106.2,1104.6,1153.0,...,630.0,2593.2,2957.20,3130.8,2971.50,1988.4,1750.8,1204.2,1276.0,1221.00


In [15]:
user_load_data.save_csv(consolidation_dataSet, 'consolidation_dataSet.csv')

save: data/2018-08-28-21-03-48_consolidation_dataSet.csv


In [16]:
# 缺值處理
fillna_dataSet = pipline.process_na_dataSet(consolidation_dataSet)
fillna_dataSet.head()

刪除未達門檻值之資料，before: 7548, after: 5120
刪除最前或最後有缺值之資料，before: 5120, after: 5118
刪除無法補值之資料，before: 5118, after: 5092


,uuid,userId,reportTime,period_1,period_2,period_3,period_4,period_5,period_6,period_7,...,period_87,period_88,period_89,period_90,period_91,period_92,period_93,period_94,period_95,period_96
0,1020170801,1,2017-08-01,907.80,701.2,964.2,817.75,813.6,1037.8,723.2,...,1577.2,4020.0,4583.25,1840.6,1320.40,1093.4,907.0,893.0,906.8,469.75
1,1020170802,1,2017-08-02,693.80,694.8,947.6,1022.60,1286.6,848.6,1360.0,...,1818.2,1599.2,1741.60,1537.0,1150.20,1271.0,1236.4,1567.8,1151.8,1392.60
2,1020170803,1,2017-08-03,974.25,1343.2,1513.0,844.80,1148.4,1326.2,1319.8,...,1571.0,4096.6,1514.00,902.0,797.00,962.8,791.4,951.4,884.2,734.80
3,1020170804,1,2017-08-04,1090.25,979.6,699.8,914.20,674.6,676.8,752.6,...,5295.8,3434.0,2591.60,3004.6,2508.75,1623.0,1172.2,1183.8,889.0,1394.00
4,1020170805,1,2017-08-05,1080.50,1151.0,1109.6,883.20,1106.2,1104.6,1153.0,...,630.0,2593.2,2957.20,3130.8,2971.50,1988.4,1750.8,1204.2,1276.0,1221.00


In [17]:
fillna_dataSet.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5092 entries, 0 to 7546
Data columns (total 99 columns):
uuid          5092 non-null object
userId        5092 non-null int32
reportTime    5092 non-null datetime64[ns]
period_1      5092 non-null float64
period_2      5092 non-null float64
period_3      5092 non-null float64
period_4      5092 non-null float64
period_5      5092 non-null float64
period_6      5092 non-null float64
period_7      5092 non-null float64
period_8      5092 non-null float64
period_9      5092 non-null float64
period_10     5092 non-null float64
period_11     5092 non-null float64
period_12     5092 non-null float64
period_13     5092 non-null float64
period_14     5092 non-null float64
period_15     5092 non-null float64
period_16     5092 non-null float64
period_17     5092 non-null float64
period_18     5092 non-null float64
period_19     5092 non-null float64
period_20     5092 non-null float64
period_21     5092 non-null float64
period_22     5092 non-nu

In [18]:
user_load_data.save_csv(fillna_dataSet, 'fillna_dataSet.csv')

save: data/2018-08-28-21-04-39_fillna_dataSet.csv


In [19]:
# 計算 最大需量、最大需量、總用電量
max_min_sum_w_dataSet = pipline.calc_peroid_max_min_sum_w(fillna_dataSet)
max_min_sum_w_dataSet.head()

,uuid,userId,reportTime,period_1,period_2,period_3,period_4,period_5,period_6,period_7,...,period_90,period_91,period_92,period_93,period_94,period_95,period_96,wMax,wMin,wSum
0,1020170801,1,2017-08-01,907.80,701.2,964.2,817.75,813.6,1037.8,723.2,...,1840.6,1320.40,1093.4,907.0,893.0,906.8,469.75,4595.2,135.50,20667.67
1,1020170802,1,2017-08-02,693.80,694.8,947.6,1022.60,1286.6,848.6,1360.0,...,1537.0,1150.20,1271.0,1236.4,1567.8,1151.8,1392.60,3382.5,176.75,21649.61
2,1020170803,1,2017-08-03,974.25,1343.2,1513.0,844.80,1148.4,1326.2,1319.8,...,902.0,797.00,962.8,791.4,951.4,884.2,734.80,4585.4,149.20,32833.36
3,1020170804,1,2017-08-04,1090.25,979.6,699.8,914.20,674.6,676.8,752.6,...,3004.6,2508.75,1623.0,1172.2,1183.8,889.0,1394.00,5295.8,137.20,24774.05
4,1020170805,1,2017-08-05,1080.50,1151.0,1109.6,883.20,1106.2,1104.6,1153.0,...,3130.8,2971.50,1988.4,1750.8,1204.2,1276.0,1221.00,3130.8,169.60,28111.41


In [20]:
user_load_data.save_csv(max_min_sum_w_dataSet, 'max_min_sum_w_dataSet.csv')

save: data/2018-08-28-21-04-49_max_min_sum_w_dataSet.csv
